In [ ]:
import requests
import pandas as pd

link = "https://servicodados.ibge.gov.br/api/v3/agregados/7392/periodos/-6/variaveis/10484?localidades=N1[all]"

requisicao = requests.get(link)
dados = requisicao.json()

# Extrair os dados relevantes
dados_organizados = []

for item in dados:
    id_variavel = item['id']
    variavel = item['variavel']
    unidade = item['unidade']
    
    for resultado in item['resultados']:
        for classificacao in resultado['classificacoes']:
            id_classificacao = classificacao['id']
            nome_classificacao = classificacao['nome']
            categoria_id, categoria_nome = list(classificacao['categoria'].items())[0]
            
            for serie in resultado['series']:
                localidade_id = serie['localidade']['id']
                localidade_nome = serie['localidade']['nome']
                nivel_id = serie['localidade']['nivel']['id']
                nivel_nome = serie['localidade']['nivel']['nome']
                
                for ano, valor in serie['serie'].items():
                    dados_organizados.append({
                        'ID Variável': id_variavel,
                        'Variável': variavel,
                        'Unidade': unidade,
                        'ID Classificação': id_classificacao,
                        'Classificação': nome_classificacao,
                        'ID Categoria': categoria_id,
                        'Categoria': categoria_nome,
                        'ID Localidade': localidade_id,
                        'Localidade': localidade_nome,
                        'ID Nível': nivel_id,
                        'Nível': nivel_nome,
                        'Ano': ano,
                        'Valor': valor
                    })

# Criar o DataFrame
df = pd.DataFrame(dados_organizados)

df.to_excel(r"C:\Users\marce\OneDrive\Arquivos Antigos\Documentos\GitHub\Python_Projetos\Datasets\teste.xlsx")

# Exibir o DataFrame
print(df)

In [5]:
import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import random
import pyodbc
from datetime import date
from faker import Faker

fake = Faker() #variavel para usar Faker

data1 = fake.date_time_between(start_date='-15y',end_date='now')

print(data1)

2021-03-26 04:53:34


In [ ]:
import pandas as pd

# Exemplo de DataFrame com nomes
data = {
    "NOME": ["João", "João", "João", "João", "João", "Maria", "Maria", "Maria", "Pedro", "Ferna", "Ferna", "Ferna", "Rober", "Rober", "Rober", "Rober", "Rober", "Rober"]
}
df = pd.DataFrame(data)

# Exemplo de DataFrame com idades
data1 = {
    "IDADE": [
        25, 25, 25, 25,  # Sequência de João
        25, 25, 25,       # Sequência de Maria
        27,               # Sequência de Pedro
        37, 37, 37,       # Sequência de Ferna
        25, 25, 25, 25, 25, 25  # Sequência de Rober
    ]
}
df1 = pd.DataFrame(data1)

# Lista para armazenar as primeiras idades de cada sequência
primeiras_idades = []

# Índice inicial para percorrer as idades
indice = 0

# Função para agrupar nomes em sequências
def agrupar_sequencias_nomes(nomes):
    sequencias = []
    if len(nomes) == 0:
        return sequencias
    
    nome_atual = nomes[0]
    inicio = 0
    
    for i, nome in enumerate(nomes):
        if nome != nome_atual:
            sequencias.append((nome_atual, inicio, i - 1))  # (nome, início, fim)
            nome_atual = nome
            inicio = i
    sequencias.append((nome_atual, inicio, len(nomes) - 1))  # Última sequência
    return sequencias

# Agrupar os nomes em sequências
sequencias_nomes = agrupar_sequencias_nomes(df["NOME"].tolist())

# Verificar se o número de sequências de nomes corresponde ao número de sequências de idades
if len(sequencias_nomes) != len(df1["IDADE"].unique()):
    raise ValueError("O número de sequências de nomes não corresponde ao número de sequências de idades.")

# Associar cada nome à primeira idade de sua sequência
for nome, inicio, fim in sequencias_nomes:
    # Adicionar a primeira idade da sequência correspondente ao nome
    primeiras_idades.append(df1["IDADE"][indice])
    
    # Avançar o índice para o início da próxima sequência de idades
    indice += (fim - inicio + 1)

# Criar o DataFrame resultante
df_resultante = pd.DataFrame({
    "NOME": [nome for nome, _, _ in sequencias_nomes],  # Nomes únicos
    "IDADE": primeiras_idades
})

print("DataFrame resultante:")
print(df_resultante)

In [12]:
#Script que gera NF, ITEMNF e VENDAS em formaato .xlsx direto na pasta para a importacao

import random
import pandas as pd
import os
from datetime import datetime, date
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import numpy as np
from faker import Faker

# Quantidade de notas fiscais

numero_inicial_nota = 1000
numero_maximo_nota = 3001
quant_maxima_itens = 100
variavel_controle_maximo_LOOP = 1300 #Valor maximo que da nota inicial dentro do loop de geracao de NF's

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

total = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()

print(total)

Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida
153
